# Segmenting and Clustering Neighborhoods in Toronto

##  Step 1: Use the BeautifulSoup package to transform the data in the table on the Wikipedia page into the pandas dataframe

#### import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
%load_ext blackcellmagic

print('Libraries imported.')

Libraries imported.


In [2]:
from bs4 import BeautifulSoup
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Fetch the html file
html_doc = requests.get(link).text


In [3]:
soup = BeautifulSoup(html_doc)
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [4]:
soup.tbody.td.contents

['M1A']

In [5]:
for child in soup.tbody.children:
    print(child)

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>


<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>


<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>


<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>


<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>


<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>


<tr>
<td>M6A</td>
<td><a href="/wiki/North_Yo

#### Create Lists to store Postcode,Borough,Neighbourhood scraping from wiki

In [6]:
Postcode = []
Borough = []
Neighborhood = []
n=0
for child in soup.tbody.children:
    if n == 0:
        n+=1
        print("The content of line {} is:".format(n), child.contents)
        Postcode.append(child.contents[1].string)
        Borough.append(child.contents[3].string)
        Neighborhood.append(child.contents[5].string)
    else:
        n+=1
        try:
            print("The content of line {} is:".format(n), child.contents)
            Postcode.append(child.contents[1].string)
            Borough.append(child.contents[3].string)        
            try:
                Neighborhood.append(child.contents[5].contents[0].contents[0])
            except:
                print("Neighborhood object has no attribute 'contents'")
                Neighborhood.append("Not assigned")
        
        except:
            print("object has no attribute 'contents'")

The content of line 1 is: ['\n', <th>Postcode</th>, '\n', <th>Borough</th>, '\n', <th>Neighbourhood
</th>]
object has no attribute 'contents'
The content of line 3 is: ['\n', <td>M1A</td>, '\n', <td>Not assigned</td>, '\n', <td>Not assigned
</td>]
Neighborhood object has no attribute 'contents'
object has no attribute 'contents'
The content of line 5 is: ['\n', <td>M2A</td>, '\n', <td>Not assigned</td>, '\n', <td>Not assigned
</td>]
Neighborhood object has no attribute 'contents'
object has no attribute 'contents'
The content of line 7 is: ['\n', <td>M3A</td>, '\n', <td><a href="/wiki/North_York" title="North York">North York</a></td>, '\n', <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>]
object has no attribute 'contents'
The content of line 9 is: ['\n', <td>M4A</td>, '\n', <td><a href="/wiki/North_York" title="North York">North York</a></td>, '\n', <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>]
object has no attribute 'c

In [7]:
Postcode[:10]

['Postcode', 'M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A']

#### Create original dataframe from lists

In [8]:
df = pd.DataFrame([Postcode, Borough, Neighborhood]).T
df


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


#### Using Pandas for data cleaning

In [9]:
columns = []
for i in df.iloc[0,:]:
    columns.append(i)
columns[2] = columns[2].strip("\n")
columns


['Postcode', 'Borough', 'Neighbourhood']

In [10]:
df.columns = columns
df = df.iloc[1:,:]
df.head(10)
df.reset_index(drop=True)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
df = df[df.Borough != "Not assigned"]
df


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [12]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == "Not assigned", df["Borough"], df["Neighbourhood"])
df

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


#### For more than one neighborhood exist in one postal code area, combine two rows into one row with the neighborhoods separated with a comma

In [13]:
df.Postcode.value_counts()[:5]

M9V    8
M8Y    8
M5V    7
M9B    5
M8Z    5
Name: Postcode, dtype: int64

In [14]:
df2 = pd.DataFrame(df[["Postcode","Neighbourhood"]].groupby(by = ["Postcode"]).apply(lambda x: ",".join(x.Neighbourhood)))
df2.columns = ["Neighbourhood"]
df2

,Neighbourhood
Postcode,
M1B,"Rouge,Malvern"
M1C,"Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Morningside,West Hill"
M1G,Woburn
M1H,Cedarbrae
M1J,Scarborough Village
M1K,"Scarborough,Ionview,Kennedy Park"
M1L,"Clairlea,Golden Mile,Oakridge"
M1M,"Cliffcrest,Cliffside,Scarborough"


In [15]:
df1 = df.iloc[:,:2]
df1.drop_duplicates(subset="Postcode",keep='first', inplace=True)
df1

C:\Users\zhunqiang\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough
3,M3A,North York
4,M4A,North York
5,M5A,Downtown Toronto
7,M6A,North York
9,M7A,Queen's Park
11,M9A,Etobicoke
12,M1B,Scarborough
15,M3B,North York
16,M4B,East York
18,M5B,Downtown Toronto


#### Merge df1 and df2 to get final dataframe

In [16]:
df_merged = df1.merge(df2, how="inner", on="Postcode")
df_merged

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,North York
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Downtown Toronto"


#### use the .shape method to print the number of rows of dataframe.

In [17]:
df_merged.shape

(103, 3)

##  Step 2: Get the latitude and the longitude coordinates of each neighborhood.

In [18]:
lat = []
lng = []
key = "" #input google api key
for postcode in df_merged.Postcode:
    URL = "https://maps.googleapis.com/maps/api/geocode/json?address={},+Toronto,+CA&key={}".format(
    postcode,key
)

    # Do the request and get the response data
    req = requests.get(URL)
    res = req.json()
    
    result = res['results'][0]
    lat.append(result['geometry']['location']['lat'])
    lng.append(result['geometry']['location']['lng'])  
    

In [19]:
df_merged["Latitude"] = lat
df_merged["Longitude"] = lng

In [20]:
df_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,North York,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Downtown Toronto",43.657162,-79.378937


##  Step 3: Explore and cluster the neighborhoods

#### Use geopy library to get the latitude and longitude values of Toronto.

In [21]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [22]:
neighborhoods = df_merged

In [23]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(
    neighborhoods["Latitude"],
    neighborhoods["Longitude"],
    neighborhoods["Borough"],
    neighborhoods["Neighbourhood"],
):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False,
    ).add_to(map_toronto)

map_toronto


#### Define Foursquare Credentials and Version

In [54]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


explore the first neighborhood in our dataframe.

In [26]:
neighborhoods.loc[0, 'Neighbourhood']

'Parkwoods'

In [27]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


get the top 100 venues that are in Parkwoods within a radius of 500 meters.

In [28]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=A2QMDS2C2KXDGBHVAX4PZG0WXASY2X3ECGXSOPTOAFQKHYER&client_secret=YFR5WAKC1KGEVAVXJTHT31B5COQBF5CAZ4DGRLHZJIRJIWGF&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c82b517351e3d13abe109dc'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean the json and structure it into a pandas dataframe.

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


#### Explore Neighborhoods in Toronto

create a function to repeat the same process to all the neighborhoods in Toronto

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [34]:
# type your answer here

toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
North York
Woodbine Gardens,Parkview Hill
Ryerson,Downtown Toronto
Glencairn
Etobicoke,Islington,Etobicoke,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,North York
Woodbine Heights
St. James Town
Humewood-Cedarvale
Etobicoke,Etobicoke,Markland Wood,Etobicoke
Scarborough,Morningside,West Hill
The Beaches
Berczy Park
York
Woburn
Leaside
Downtown Toronto
Downtown Toronto
Cedarbrae
Hillcrest Village
Bathurst Manor,North York,Wilson Heights
Thorncliffe Park
Adelaide,King,Downtown Toronto
Dovercourt Village,West Toronto
Scarborough Village
North York,Henry Farm,North York
Northwood Park,York University
East Toronto
Downtown Toronto,Toronto Islands,Union Station
Little Portugal,Trinity
Scarborough,Ionview,Kennedy Park
Bayview Village
CFB Toronto,North York
East Toronto,Riverdale
Design Exchange,Toronto Dominion Centre
West Toro

In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(2499, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Downtown Toronto",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,Scarborough,Milliken,Scarborough",3,3,3,3,3,3
"Alderwood,Long Branch",8,8,8,8,8,8
"Bathurst Manor,North York,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,North York",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
"Birch Cliff,Scarborough",100,100,100,100,100,100


In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


### Analyze Each Neighborhood

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0

In [39]:
toronto_onehot.shape

(2499, 278)

 group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Down

In [41]:
toronto_grouped.shape

(89, 278)

 print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 15

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Downtown Toronto----
                  venue  freq
0           Coffee Shop  0.06
1                  Café  0.05
2       Thai Restaurant  0.04
3            Steakhouse  0.04
4                   Bar  0.04
5                 Hotel  0.03
6          Burger Joint  0.03
7                   Gym  0.03
8   American Restaurant  0.03
9            Restaurant  0.03
10     Asian Restaurant  0.03
11               Bakery  0.03
12            Gastropub  0.02
13         Concert Hall  0.02
14          Pizza Place  0.02


----Agincourt----
                         venue  freq
0                 Skating Rink  0.25
1                       Lounge  0.25
2               Breakfast Spot  0.25
3               Clothing Store  0.25
4           Mexican Restaurant  0.00
5          Monument / Landmark  0.00
6   Modern European Restaurant  0.00
7            Mobile Phone Shop  0.00
8           Miscellaneous Shop  0.00
9    Middle Eastern Restaurant  0.00
10                 Yoga Studio  0.00
11               

                  venue  freq
0        Clothing Store  0.08
1   American Restaurant  0.04
2            Steakhouse  0.03
3                 Hotel  0.03
4        Cosmetics Shop  0.03
5              Tea Room  0.03
6        Breakfast Spot  0.03
7                 Plaza  0.03
8           Coffee Shop  0.03
9       Bubble Tea Shop  0.02
10               Bakery  0.02
11         Concert Hall  0.02
12              Theater  0.02
13     Asian Restaurant  0.02
14            Gastropub  0.02


----Design Exchange,Toronto Dominion Centre----
                  venue  freq
0           Coffee Shop  0.13
1                 Hotel  0.09
2                  Café  0.08
3   American Restaurant  0.04
4             Gastropub  0.03
5         Deli / Bodega  0.03
6            Restaurant  0.03
7                   Gym  0.02
8          Burger Joint  0.02
9           Pizza Place  0.02
10   Italian Restaurant  0.02
11   Seafood Restaurant  0.02
12           Steakhouse  0.02
13  Japanese Restaurant  0.02
14                  

                         venue  freq
0                  Pizza Place  0.25
1                         Park  0.25
2          Japanese Restaurant  0.25
3                          Pub  0.25
4                Metro Station  0.00
5   Modern European Restaurant  0.00
6            Mobile Phone Shop  0.00
7           Miscellaneous Shop  0.00
8    Middle Eastern Restaurant  0.00
9           Mexican Restaurant  0.00
10                 Men's Store  0.00
11                       Motel  0.00
12    Mediterranean Restaurant  0.00
13              Medical Center  0.00
14              Massage Studio  0.00


----Harbourfront,Regent Park----
                 venue  freq
0          Coffee Shop  0.17
1                 Park  0.06
2               Bakery  0.06
3                 Café  0.06
4                  Pub  0.06
5   Mexican Restaurant  0.04
6              Theater  0.04
7       Breakfast Spot  0.04
8       Farmers Market  0.02
9       Chocolate Shop  0.02
10       Historic Site  0.02
11             Brewery  0

                         venue  freq
0               Massage Studio  0.25
1           Falafel Restaurant  0.25
2                  Coffee Shop  0.25
3                          Bar  0.25
4                  Yoga Studio  0.00
5    Middle Eastern Restaurant  0.00
6          Monument / Landmark  0.00
7   Modern European Restaurant  0.00
8            Mobile Phone Shop  0.00
9           Miscellaneous Shop  0.00
10               Metro Station  0.00
11          Mexican Restaurant  0.00
12               Movie Theater  0.00
13                 Men's Store  0.00
14    Mediterranean Restaurant  0.00


----Parkdale,Roncesvalles----
                          venue  freq
0                Breakfast Spot  0.12
1                     Gift Shop  0.12
2                     Bookstore  0.06
3                  Dessert Shop  0.06
4                  Burger Joint  0.06
5                 Movie Theater  0.06
6   Eastern European Restaurant  0.06
7                           Bar  0.06
8                          Bank  0

                  venue  freq
0     Indian Restaurant  0.12
1           Yoga Studio  0.06
2   Housing Development  0.06
3           Pizza Place  0.06
4                  Park  0.06
5           Coffee Shop  0.06
6        Discount Store  0.06
7        Sandwich Place  0.06
8          Burger Joint  0.06
9          Liquor Store  0.06
10          Supermarket  0.06
11                 Bank  0.06
12             Pharmacy  0.06
13      Warehouse Store  0.06
14        Grocery Store  0.06


----Victoria Village----
                         venue  freq
0                  Pizza Place   0.2
1                 Hockey Arena   0.2
2        Portuguese Restaurant   0.2
3                  Coffee Shop   0.2
4                 Intersection   0.2
5     Mediterranean Restaurant   0.0
6               Medical Center   0.0
7                  Men's Store   0.0
8                Metro Station   0.0
9               Massage Studio   0.0
10                 Music Store   0.0
11   Middle Eastern Restaurant   0.0
12          

put that into a pandas dataframe

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display the top 10 venues for each neighborhood.

In [44]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Adelaide,King,Downtown Toronto",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Restaurant,Gym,Burger Joint,Hotel,Asian Restaurant,Bakery,American Restaurant,Sushi Restaurant,Japanese Restaurant,Concert Hall
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Drugstore,Department Store,Eastern European Restaurant,Electronics Store
2,"Agincourt North,Scarborough,Milliken,Scarborough",Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Deli / Bodega,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,"Alderwood,Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,Gift Shop,Cuban Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
4,"Bathurst Manor,North York,Wilson Heights",Coffee Shop,Shopping Mall,Chinese Restaurant,Deli / Bodega,Sushi Restaurant,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Fast Food Restaurant,Supermarket,Pizza Place,Pharmacy,Sandwich Place


#### Cluster Neighborhoods

In [45]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 0])

In [46]:
kmeans.labels_

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2,
       2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2])

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted.rename({'Neighborhood': 'Neighbourhood'}, axis=1,inplace=True)
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how="inner")

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Fast Food Restaurant,Food & Drink Shop,Park,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Portuguese Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store,Donut Shop,Deli / Bodega,Drugstore
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2,Coffee Shop,Pub,Café,Park,Bakery,Breakfast Spot,Mexican Restaurant,Theater,Chocolate Shop,Spa,Beer Store,Brewery,Shoe Store,Restaurant,Gym / Fitness Center
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2,Clothing Store,Furniture / Home Store,Event Space,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Doner Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Donut Shop
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Diner,College Cafeteria,Gym,Japanese Restaurant,Burger Joint,Yoga Studio,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Liquor Store,Portuguese Restaurant,Café


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
    toronto_merged["Latitude"],
    toronto_merged["Longitude"],
    toronto_merged["Neighbourhood"],
    toronto_merged["Cluster Labels"],
):
    label = folium.Popup(str(poi) + " Cluster " + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7,
    ).add_to(map_clusters)

map_clusters


#### Examine Clusters

Cluster 1

In [49]:
toronto_merged.loc[
    toronto_merged["Cluster Labels"] == 0,
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))],
]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North York,0,Fast Food Restaurant,Food & Drink Shop,Park,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
10,North York,0,Park,Pizza Place,Japanese Restaurant,Pub,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Dance Studio,Donut Shop,Drugstore,Dumpling Restaurant
21,York,0,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Gourmet Shop,Empanada Restaurant,Grocery Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Discount Store
40,North York,0,Park,Bus Stop,Airport,Electronics Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
46,North York,0,Park,Grocery Store,Bank,Shopping Mall,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store,Donut Shop,Doner Restaurant,Ethiopian Restaurant,Dog Run,Discount Store
49,North York,0,Park,Bakery,Construction & Landscaping,Basketball Court,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
61,Central Toronto,0,Park,Bus Line,Swim School,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Department Store,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
77,Etobicoke,0,Park,Mobile Phone Shop,Pizza Place,Bus Line,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Deli / Bodega,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
91,Downtown Toronto,0,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dance Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant
98,Etobicoke,0,Park,Smoke Shop,River,College Gym,College Rec Center,Empanada Restaurant,Electronics Store,College Cafeteria,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Discount Store


Cluster 2

In [50]:
toronto_merged.loc[
    toronto_merged["Cluster Labels"] == 1,
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))],
]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
22,Scarborough,1,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Department Store,Eastern European Restaurant,Electronics Store,Empanada Restaurant
32,Scarborough,1,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Deli / Bodega,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
83,Central Toronto,1,Summer Camp,Playground,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Deli / Bodega,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
85,Scarborough,1,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Deli / Bodega,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


Cluster 3

In [51]:
toronto_merged.loc[
    toronto_merged["Cluster Labels"] == 2,
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))],
]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,North York,2,Portuguese Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store,Donut Shop,Deli / Bodega,Drugstore
2,Downtown Toronto,2,Coffee Shop,Pub,Café,Park,Bakery,Breakfast Spot,Mexican Restaurant,Theater,Chocolate Shop,Spa,Beer Store,Brewery,Shoe Store,Restaurant,Gym / Fitness Center
3,North York,2,Clothing Store,Furniture / Home Store,Event Space,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Doner Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Donut Shop
4,Queen's Park,2,Coffee Shop,Diner,College Cafeteria,Gym,Japanese Restaurant,Burger Joint,Yoga Studio,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Liquor Store,Portuguese Restaurant,Café
6,Scarborough,2,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
7,North York,2,Restaurant,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Electronics Store,Pizza Place,Gym / Fitness Center,Japanese Restaurant,Grocery Store,Steakhouse,Baseball Field,Middle Eastern Restaurant,Basketball Court,Plaza
53,North York,2,Restaurant,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Electronics Store,Pizza Place,Gym / Fitness Center,Japanese Restaurant,Grocery Store,Steakhouse,Baseball Field,Middle Eastern Restaurant,Basketball Court,Plaza
59,North York,2,Restaurant,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Electronics Store,Pizza Place,Gym / Fitness Center,Japanese Restaurant,Grocery Store,Steakhouse,Baseball Field,Middle Eastern Restaurant,Basketball Court,Plaza
60,North York,2,Restaurant,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Electronics Store,Pizza Place,Gym / Fitness Center,Japanese Restaurant,Grocery Store,Steakhouse,Baseball Field,Middle Eastern Restaurant,Basketball Court,Plaza
66,North York,2,Restaurant,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Electronics Store,Pizza Place,Gym / Fitness Center,Japanese Restaurant,Grocery Store,Steakhouse,Baseball Field,Middle Eastern Restaurant,Basketball Court,Plaza


Cluster 4

In [52]:
toronto_merged.loc[
    toronto_merged["Cluster Labels"] == 3,
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))],
]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
45,North York,3,Cafeteria,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store,Department Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


Compare the difference between different clusters

In [53]:
toronto_merged.groupby("Cluster Labels").agg(lambda x: x.value_counts().index[0]).loc[
    :,
    [
        "1st Most Common Venue",
        "2nd Most Common Venue",
        "3rd Most Common Venue",
        "4th Most Common Venue",
        "5th Most Common Venue",
    ],
]


,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Cluster Labels,,,,,
0,Park,Trail,Playground,Dim Sum Restaurant,Dog Run
1,Playground,Playground,Women's Store,Dessert Shop,Dim Sum Restaurant
2,Coffee Shop,Café,Coffee Shop,Coffee Shop,Sandwich Place
3,Cafeteria,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner
